In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,11790
2,Huelva,Confirmados PDIA 14 días,723
3,Huelva,Tasa PDIA 14 días,"141,55847461443426"
4,Huelva,Confirmados PDIA 7 días,458
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,20
629,Municipio de Huelva sin especificar,Total Confirmados,155
630,Municipio de Huelva sin especificar,Curados,115


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  11790.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  3356.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1072 personas en los últimos 7 días 

Un positivo PCR cada 633 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,11790.0,458.0,723.0,510743.0,89.673280,141.558475,272.0
Huelva-Costa,6726.0,303.0,478.0,288115.0,105.166340,165.905975,190.0
Huelva (capital),3356.0,134.0,227.0,143663.0,93.273842,158.008673,76.0
Condado-Campiña,3575.0,101.0,166.0,155057.0,65.137337,107.057405,57.0
Isla Cristina,547.0,80.0,101.0,21264.0,376.222724,474.981189,55.0
Moguer,422.0,32.0,43.0,22088.0,144.875045,194.675842,17.0
Cartaya,449.0,19.0,36.0,19974.0,95.123661,180.234305,16.0
Palma del Condado (La),581.0,26.0,42.0,10761.0,241.613233,390.298299,15.0
Lepe,1003.0,21.0,39.0,27431.0,76.555722,142.174912,15.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cerro de Andévalo (El),57.0,12.0,19.0,2364.0,507.614213,803.722504,5.0
Almendro (El),21.0,5.0,5.0,826.0,605.326877,605.326877,4.0
Cumbres Mayores,19.0,7.0,10.0,1755.0,398.860399,569.800570,0.0
Isla Cristina,547.0,80.0,101.0,21264.0,376.222724,474.981189,55.0
Villablanca,69.0,2.0,13.0,2848.0,70.224719,456.460674,0.0
Palma del Condado (La),581.0,26.0,42.0,10761.0,241.613233,390.298299,15.0
Lucena del Puerto,111.0,5.0,7.0,3371.0,148.323939,207.653515,0.0
Moguer,422.0,32.0,43.0,22088.0,144.875045,194.675842,17.0
Cartaya,449.0,19.0,36.0,19974.0,95.123661,180.234305,16.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Villablanca,69.0,2.0,13.0,2848.0,70.224719,456.460674,0.0,0.153846
Almonte,453.0,3.0,9.0,24191.0,12.401306,37.203919,2.0,0.333333
Palos de la Frontera,271.0,10.0,20.0,11289.0,88.581805,177.163611,3.0,0.500000
Nerva,89.0,1.0,2.0,5235.0,19.102197,38.204394,1.0,0.500000
Villalba del Alcor,105.0,1.0,2.0,3338.0,29.958059,59.916117,0.0,0.500000
Cartaya,449.0,19.0,36.0,19974.0,95.123661,180.234305,16.0,0.527778
Lepe,1003.0,21.0,39.0,27431.0,76.555722,142.174912,15.0,0.538462
Huelva (capital),3356.0,134.0,227.0,143663.0,93.273842,158.008673,76.0,0.590308
Condado-Campiña,3575.0,101.0,166.0,155057.0,65.137337,107.057405,57.0,0.608434
